<a href="https://colab.research.google.com/github/pallavi-allada/experiments-with-data/blob/master/Inria_Preprocessing_SlidingWindowPatches_250.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Inria Aerial Satellite Image Labelling**
The main aim of this challenge is to be able to classify pixels as 'building' or 'not building'. This is a clear case of semantic segmentation. We have180 aerial images from various cities with a resolution of 5000 X 5000, along with the segmented masks of 5000 X 5000. Our goal is to classify each of the pixels in the test image and generate a masks for the test images too.

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash
!pip install PyDrive

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import tarfile,re,os,pathlib,shutil

import fastai
from fastai.vision import *
import torch
import torchvision.transforms.functional as F

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Download Datasets

Downloading the dataset images from the URL mentioned in the inria site using curl cmd. This command downloads the dataset as 7z parts and then unzips it under the folder AerialImageDataset.

In [0]:
# Do not execute !! Already downloaded datasets !
!curl -k https://files.inria.fr/aerialimagelabeling/getAerial.sh | bash

Remove dependent 7z parts and zip file once unzipping is completed to clear some storage space.

In [0]:
!rm -rf /content/aerialimagelabeling*
!rm -rf /content/NEW2-AerialImageDataset.zip

## Preprocess the data 
Since each image is of a high dimension and we only have 180 training images, we can take 2 approaches - 
1) Resize the image to lower dimensions like 250 X 250 and then train the network on the resized images.
2) Slice each image of 5k X 5k over a sliding window stride of 250 to generate an image of size 250 X 250 and train the network on the patches. This approach generates ~400 image patches for each 5k X 5k image.

In [0]:
img_path = Path('/content/AerialImageDataset/train/images')
mask_path = Path('/content/AerialImageDataset/train/gt')
test_path = Path('/content/AerialImageDataset/test/images')
valid_path = Path('/content/AerialImageDataset/valid')

images_patch_path = Path('/content/AerialImagePatches/train')
masks_patch_path = Path('/content/AerialImagePatches/masks')
tests_patch_path = Path('/content/AerialImagePatches/test')
valids_patch_path = Path('/content/AerialImagePatches/valid')

Split train to valid by moving the first 5 images from each city to valid folder

In [0]:
#move first five images from each city for valid hold outs

def train_valid_split(src: str, dst: str):
  if not os.path.isdir(dst):
    dst.mkdir(parents=True, exist_ok=True)
  for file in os.listdir(src):
    if re.match("[a-zA-Z]*[1-5]{1}\.tif", file) or re.match("[a-zA-Z]*-\w[1-5]{1}\.tif", file):
      shutil.move(src/file, dst/file)
        
train_valid_split(img_path,valid_path)

Clean up any previously created folder for AerialImagePatches.

In [0]:
!rm -rf AerialImagePatches

Functions to generate patches of 250 X 250 from 5k X 5k images and save it for training model.

In [9]:
orig_images = ImageList.from_folder(img_path)
orig_valids = ImageList.from_folder(valid_path)
orig_masks = ImageList.from_folder(mask_path)
orig_tests = ImageList.from_folder(test_path)

get_patch_file_name = lambda pth,x,y: pth/f'{x.stem.split("/")[-1]}_{y}.tif'

channels = 3
size = 250
step = 250


def generate_patches(image, i, dest_img_path, channels:int, size:int, step:int):
  img = open_image(image)
  patches = img.data.unfold(0, channels, channels).unfold(1, size, step).unfold(2, size, step)
  cnt=1
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
      for k in range(patches.shape[2]):
        patch = patches[i][j][k]
        imgpil = F.to_pil_image(patch)
        filname = get_patch_file_name(dest_img_path, image, cnt)
        cnt=cnt+1
        imgpil.save(filname)

# create smaller patch sets 
sets = [(orig_images,images_patch_path,channels,size,step),(orig_valids,valids_patch_path,channels,size,step),(orig_masks, masks_patch_path,channels,size,step),(orig_tests,tests_patch_path,channels,size,step)]
for img_lst,dst_path,channels,size,step in sets:
  if not dst_path.exists():
    dst_path.mkdir(parents=True, exist_ok=True)
  parallel(partial(generate_patches, dest_img_path=dst_path, channels=channels, size=size, step=step), img_lst.items)

        

Now, the training set of 180 images of 5k x 5k is converted into a training set of 60450 patches of 250 x 250 each, along with corresponding 60450 patches for the masks as well. Valid set contains 9750 patches of 250 x 250.

In [0]:
len(get_image_files(img_path)),len(get_image_files(mask_path)),len(get_image_files(valid_path))

In [0]:
len(get_image_files(images_patch_path)),len(get_image_files(masks_patch_path)), len(get_image_files(valids_patch_path))


Function to create tar file for moving to Drive for future use. This helps in avoiding redundant preprocessing each time.

In [0]:
train_source_dir = '/content/AerialImagePatches/train'
valid_source_dir = '/content/AerialImagePatches/valid'
mask_source_dir = '/content/AerialImagePatches/masks'
test_source_dir = '/content/AerialImagePatches/test'

def make_tarfile(output_filename, source_dir):
  with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(source_dir, arcname=os.path.basename(source_dir))

In [0]:
make_tarfile('train.tar', train_source_dir)

In [0]:
make_tarfile('masks.tar', mask_source_dir)

In [0]:
make_tarfile('valid.tar', valid_source_dir)

In [0]:
# Tar test folder only when needed as Drive only has 15GB by default
make_tarfile('test.tar', test_source_dir)

Function to upload to drive

In [0]:
# Create GoogleDrive instance with authenticated GoogleAuth instance.
drive = GoogleDrive(gauth)

def upload_to_drive(file, title):
  uploaded = drive.CreateFile({'title': title})
  uploaded.SetContentFile(file)
  uploaded.Upload()
  print('Uploaded file %s with ID %s'%(file, uploaded.get('id')))


In [0]:
upload_to_drive("train.tar","train.tar")

In [0]:
upload_to_drive("masks.tar","masks.tar")

In [0]:
upload_to_drive("valid.tar","valid.tar")

In [0]:
upload_to_drive("test.tar","test.tar")